In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes 
import seaborn as sns
import plotly.express as px
import re

In [2]:
# Importanmos el archivo a usar
df = pd.read_csv("./properatti.csv")

In [3]:
df.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [4]:
print("--------------------------------")
print("Shape", df.shape)
print("--------------------------------")
print("Columnas y sus tipos", df.dtypes)
print("--------------------------------")

--------------------------------
Shape (121220, 26)
--------------------------------
Columnas y sus tipos Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description         

In [5]:
#Como vemos, da igual, por lo que pasamos a borrarla.
df = df.drop(["Unnamed: 0"], axis=1)
df.sample(2)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
665,sell,house,Bella Vista,|Argentina|Bs.As. G.B.A. Zona Norte|San Miguel...,Argentina,Bs.As. G.B.A. Zona Norte,3436061.0,"-34.5789658,-58.6903991",-34.578966,-58.690399,...,95.0,475.000000,1000.000000,NaN,NaN,NaN,http://www.properati.com.ar/15fke_venta_casa_b...,CODIGO: 827-194 ubicado en: Río colorado 2695 ...,LINDA CASA EN ESQUINA!,https://thumbs4.properati.com/0/EVN8zgagsQcKLB...
15688,sell,apartment,Lanús Oeste,|Argentina|Bs.As. G.B.A. Zona Sur|Lanús|Lanús ...,Argentina,Bs.As. G.B.A. Zona Sur,NaN,"-34.7011240308,-58.3959540847",-34.701124,-58.395954,...,69.0,2094.594595,2246.376812,NaN,3.0,2300.0,http://www.properati.com.ar/16vtm_venta_depart...,Venta de Departamento 3 AMBIENTES en Lanús Oes...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/1/kE4kb9Fc6iMctd...


In [6]:
#Borro la columna "operation" porque todas son ventas.
df = df.drop(["operation"], axis=1)
df.head(3)

,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [7]:
# Drop column "lat-lon" porque es repetitiva
df = df.drop(["lat-lon"], axis=1)
df

,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat,lon,price,currency,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,-34.661824,-58.508839,62000.0,USD,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,-34.903883,-57.964330,150000.0,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,-34.652262,-58.522982,72000.0,USD,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,-34.647797,-58.516424,95000.0,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,-38.002626,-57.549447,64000.0,USD,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,870000.0,USD,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,498000.0,USD,...,360.0,1383.333333,1383.333333,NaN,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,-34.570639,-58.475596,131500.0,USD,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,95900.0,USD,...,48.0,1997.916667,1997.916667,NaN,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...


In [8]:
# Deseamos conocer el porcentaje de nulos por columna;
porcentaje_nulos_por_columna= (pd.isna(df).sum()/len(df))*100
round(porcentaje_nulos_por_columna.sort_values(ascending=False),2)

floor                         93.48
expenses                      88.23
rooms                         60.91
price_usd_per_m2              43.39
lat                           42.53
lon                           42.53
surface_total_in_m2           32.44
price_per_m2                  27.69
currency                      16.84
price_aprox_usd               16.84
price                         16.84
price_aprox_local_currency    16.84
surface_covered_in_m2         16.42
geonames_id                   15.44
image_thumbnail                2.57
place_name                     0.02
description                    0.00
title                          0.00
properati_url                  0.00
state_name                     0.00
country_name                   0.00
place_with_parent_names        0.00
property_type                  0.00
dtype: float64

In [9]:
# Revisamos el porcentaje de propiedades que hay en cada región.
state_name_mix = (df["state_name"].value_counts()/len(df)).reset_index()
state_name_mix['state_name'] = (state_name_mix['state_name']*100).round(1) 
state_name_mix.columns = ['state_name', 'mix']
state_name_mix

,state_name,mix
0,Capital Federal,26.7
1,Bs.As. G.B.A. Zona Norte,21.1
2,Bs.As. G.B.A. Zona Sur,11.5
3,Córdoba,10.0
4,Santa Fe,8.4
5,Buenos Aires Costa Atlántica,8.3
6,Bs.As. G.B.A. Zona Oeste,7.7
7,Buenos Aires Interior,1.9
8,Río Negro,0.7
9,Neuquén,0.6


In [50]:
# Graficamos la cantidad de propiedades por provincia

fig = px.bar(data_frame = state_name_mix[state_name_mix["mix"] >1.8], x='state_name' , y='mix', text='mix' , width=500, height=500)

fig.update_layout(
    title='<b>Propiedades por Provincia</b>',
    xaxis_tickfont_size=10,
    yaxis=dict(title='% del total',titlefont_size=14,tickfont_size=10),
    title_x=0.5)
fig.update_traces(textposition='inside', textangle = 0, insidetextanchor='middle')

fig.show()

In [10]:
#Calculamos la cantidad de datos que van a quedar fuera en el nuevo df por tener pocos datos.
state_name_mix_menor_1_9 = state_name_mix[state_name_mix["mix"]<1.9]["state_name"].tolist()

df.loc[df["state_name"].isin(state_name_mix_menor_1_9),:].shape

(5532, 23)

In [12]:
#Sort dataframe by columns

df_column_sorted = {
    "floor": df["floor"],
    "expenses": df["expenses"],
    "rooms": df["rooms"],
    "price_usd_per_m2": df["price_usd_per_m2"],
    "lon": df["lon"],
    "lat": df["lat"],
    "surface_total_in_m2": df["surface_total_in_m2"],
    "currency": df["currency"],
    "price_aprox_usd": df["price_aprox_usd"],
    "price_aprox_local_currency": df["price_aprox_local_currency"],
    "price": df["price"],
    "surface_covered_in_m2": df["surface_covered_in_m2"],
    "place_name": df["place_name"],
    "geonames_id": df["geonames_id"],
    "state_name": df["state_name"],
    "title": df["title"],
    "description": df["description"],
    "property_type": df["property_type"]
}

df = pd.DataFrame(df_column_sorted)

In [13]:
df = df.set_index("state_name")

In [14]:
#Antes de hacer el count de abajo, quiero, en esta línea, dejar sólo los registros iguales a:
state_name_list_ok = ["Capital Federal",
"Bs.As. G.B.A. Zona Norte",
"Bs.As. G.B.A. Zona Sur",
"Córdoba",
"Santa Fe",
"Buenos Aires Costa Atlántica",
"Bs.As. G.B.A. Zona Oeste",
"Buenos Aires Interior"]	

In [15]:
df.shape

(121220, 17)

In [16]:
df.loc[state_name_list_ok, :].shape

(115688, 17)

In [17]:
# df con los datos de los 8 state_name más importantes.
df = df.loc[state_name_list_ok, :]

In [18]:
# Vemos los valores de mean y max de apartment porque nos habíamos encontrado con números de 2mil para arriba;
df.groupby(["state_name", "property_type"]).describe().loc["Capital Federal"]

floor                                                      \
                count       mean        std  min  25%  50%    75%     max   
property_type                                                               
PH              155.0   2.251613   2.211521  1.0  1.0  1.0   3.00    16.0   
apartment      2838.0  13.243834  84.696737  1.0  2.0  4.0   7.00  2509.0   
house             8.0  10.875000   7.395703  2.0  6.0  9.5  14.00    26.0   
store             8.0   9.375000  10.979689  1.0  2.0  3.0  15.25    31.0   

              expenses               ... surface_covered_in_m2            \
                 count         mean  ...                   75%       max   
property_type                        ...                                   
PH               182.0   962.730769  ...                 115.0    1183.0   
apartment       6196.0  6634.878631  ...                  79.0  187000.0   
house             12.0  2675.916667  ...                 320.0    8175.0   
store            202.0  2896.549505  ...                 231.0   10958.0   

              geonames_id                                                     \
                    count          mean            std        min        25%   
property_type                                                                  
PH                 1766.0  3.529494e+06  556658.475395  3427366.0  3429422.0   
apartment         25950.0  3.530139e+06  555737.593268  3427366.0  3429760.0   
house              2021.0  3.469355e+06  345993.449956  3427366.0  3430114.0   
store              1293.0  3.578327e+06  675300.520405  3427366.0  3429595.0   

                                                
                     50%        75%        max  
property_type                                   
PH             3430787.0  3433955.0  6693230.0  
apartment      3433775.0  3436003.0  6693230.0  
house          3433910.0  3433955.0  6693230.0  
store          3430787.0  3435548.0  6693230.0  

[4 rows x 96 columns]

In [19]:
#Vemos la cantidad de no nulos en la columna floor para todos los datos.
df.loc[df["floor"].notnull()].describe()

,floor,expenses,rooms,price_usd_per_m2,lon,lat,surface_total_in_m2,price_aprox_usd,price_aprox_local_currency,price,surface_covered_in_m2,geonames_id
count,7615.000000,1027.000000,5441.000000,3208.000000,5905.000000,5905.000000,3595.000000,6.795000e+03,6.795000e+03,6.795000e+03,6898.000000,6.812000e+03
mean,17.403152,2522.842259,2.647307,2340.929008,-59.683618,-33.928174,81.444228,2.251617e+05,3.972866e+06,3.527507e+05,99.895332,3.635403e+06
std,116.985284,2481.155954,1.271013,868.641196,2.270634,1.488243,245.697695,3.037610e+05,5.359711e+06,6.354222e+05,325.831184,5.406267e+05
min,1.000000,1.000000,1.000000,29.906933,-75.678931,-38.583214,10.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,3.427326e+06
25%,1.000000,1200.000000,2.000000,1857.142857,-60.639286,-34.618785,45.000000,9.200000e+04,1.623294e+06,1.020000e+05,44.000000,3.429651e+06
50%,3.000000,1800.000000,3.000000,2239.436620,-58.478612,-34.589013,60.000000,1.380000e+05,2.434941e+06,1.500000e+05,60.000000,3.433955e+06
75%,6.000000,2869.000000,3.000000,2787.656423,-58.423104,-32.957491,84.000000,2.250000e+05,3.970012e+06,2.710000e+05,93.000000,3.838574e+06
max,2509.000000,19000.000000,22.000000,19800.000000,-56.674305,4.545843,9563.000000,3.900000e+06,6.881355e+07,1.168104e+07,12731.000000,6.942539e+06


In [20]:
df["floor"].sort_values(ascending=False).head(40)

state_name
Capital Federal             2509.0
Bs.As. G.B.A. Zona Norte    2177.0
Capital Federal             2103.0
Bs.As. G.B.A. Zona Norte    1609.0
Bs.As. G.B.A. Zona Norte    1609.0
Bs.As. G.B.A. Zona Norte    1609.0
Bs.As. G.B.A. Zona Norte    1609.0
Bs.As. G.B.A. Zona Norte    1608.0
Bs.As. G.B.A. Zona Norte    1608.0
Bs.As. G.B.A. Zona Norte    1607.0
Bs.As. G.B.A. Zona Norte    1606.0
Bs.As. G.B.A. Zona Norte    1602.0
Bs.As. G.B.A. Zona Norte    1512.0
Bs.As. G.B.A. Zona Norte    1504.0
Bs.As. G.B.A. Zona Norte    1504.0
Bs.As. G.B.A. Zona Norte    1450.0
Bs.As. G.B.A. Zona Norte    1420.0
Bs.As. G.B.A. Zona Norte    1407.0
Bs.As. G.B.A. Zona Norte    1404.0
Bs.As. G.B.A. Zona Norte    1403.0
Bs.As. G.B.A. Zona Norte    1401.0
Bs.As. G.B.A. Zona Norte    1379.0
Bs.As. G.B.A. Zona Norte    1306.0
Bs.As. G.B.A. Zona Norte    1305.0
Bs.As. G.B.A. Zona Norte    1304.0
Bs.As. G.B.A. Zona Norte    1301.0
Bs.As. G.B.A. Zona Norte    1291.0
Bs.As. G.B.A. Zona Norte    1208.0
Bs.As. G.

In [21]:
#Cantidad de datos no nulos en floor.
df["floor"].count()

7615

In [22]:
# Buscamos el dato del edificio más grande de arg que tiene 54 pisos.
df["floor"][df["floor"] > 54].count()

180

In [23]:
df.reset_index(inplace=True)

In [24]:
# Rescatamos el index de todos los que son mayores a 54.
indices_mayor_54_floor = df[df["floor"] > 54].index

In [25]:
indices_mayor_54_floor = indices_mayor_54_floor.tolist()

In [26]:
# porcentaje de los valores outliers de floor mayores a 54.as_integer_ratio
180/7615*100

2.3637557452396587

In [27]:
# La cantidad de datos erróneos en la columna floor respecto a la cantidad total de datos.
180/df.shape[0]*100

0.1555908996611576

In [28]:
# Imputamos los datos mayores a 54 de la columna floor por apartment con NAN.
df["floor"].where(df["floor"] < 54, np.NAN, inplace=True)

In [29]:
# Verificamos que los 180 datos que tenían datos erróneos en la columna floor, fueron reemplazados por NAN.
df[df.index.isin(indices_mayor_54_floor)].head(3)

,state_name,floor,expenses,rooms,price_usd_per_m2,lon,lat,surface_total_in_m2,currency,price_aprox_usd,price_aprox_local_currency,price,surface_covered_in_m2,place_name,geonames_id,title,description,property_type
984,Capital Federal,NaN,NaN,2.0,NaN,-58.479707,-34.566330,NaN,USD,175000.0,3087787.5,175000.0,65.0,Villa Urquiza,3433775.0,2 AMB BALCÓN CON PARRILLA - EDIFICIO CON AMENI...,Corredor Responsable: Rebeca Stern - C.U.C.I.C...,apartment
1269,Capital Federal,NaN,NaN,3.0,NaN,-58.472350,-34.575419,NaN,NaN,NaN,NaN,NaN,NaN,Belgrano,3436077.0,"LA PAMPA AL 4300, Exclusivo Desarrollo en Pesos",Corredor Responsable: Raul Hector Merlo - CUCI...,apartment
1270,Capital Federal,NaN,NaN,4.0,NaN,-58.472617,-34.575560,NaN,NaN,NaN,NaN,NaN,NaN,Belgrano,3436077.0,LA PAMPA 4300 AIR PAMPA POZO EN PESOS,Corredor Responsable: Raul Hector Merlo - CUCI...,apartment


In [30]:
# Pensamos en imputar los datos que son casas con valores de 0, porque tienen piso cero, de ahí empezamos a hacer el siguiente análisis:

# Contamos la cantidad de registros que son casas.
df_house = df.loc[df["property_type"] == "house"]
# print("Cantidad de registros que son casas del df: ", df_house, sep="")

pd.DataFrame(df_house)

,state_name,floor,expenses,rooms,price_usd_per_m2,lon,lat,surface_total_in_m2,currency,price_aprox_usd,price_aprox_local_currency,price,surface_covered_in_m2,place_name,geonames_id,title,description,property_type
13,Capital Federal,NaN,NaN,NaN,875.000000,-58.362056,-34.617719,80.0,USD,70000.00,1235115.00,70000.0,50.0,Puerto Madero,3429760.0,FLOAT LIFE - CASA FLOTANTE - UBICABLE EN CLUBS...,ESTRUCTURA ECOLOGICA FLOTANTEIDEAL PARA UBICAR...,house
15,Capital Federal,NaN,NaN,NaN,1091.269841,-58.410702,-34.636321,504.0,USD,550000.00,9704475.00,550000.0,324.0,Boedo,3436003.0,Casa impecable refaccionada! 6 ambientes.,Se encuentra dentro del Distrito Tecnológico. ...,house
26,Capital Federal,NaN,NaN,NaN,944.444444,NaN,NaN,180.0,USD,170000.00,2999565.00,170000.0,165.0,Capital Federal,3433955.0,"Casa 180m² con 2 Plantas en Navarro, Julián 29...","Casa en 2 Plantas en BÂ° La Merced, a Metros d...",house
27,Capital Federal,NaN,NaN,NaN,658.602151,NaN,NaN,744.0,USD,490000.00,8645805.00,490000.0,350.0,Capital Federal,3433955.0,Casa 744m² con 2 Plantas en Del Valle Iberluce...,"Lote: 744,76 m. (Frente: 17,32 x 43 de fondo) ...",house
28,Capital Federal,NaN,NaN,NaN,611.111111,NaN,NaN,720.0,USD,440000.00,7763580.00,440000.0,261.0,Capital Federal,3433955.0,"Casa 720m² con Plantas en Palma, Diego 1800, A...",PB. Liv y com con chimenea con salida a jardin...,house
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115678,Buenos Aires Interior,NaN,NaN,5.0,400.801603,-59.100573,-37.313734,499.0,USD,200000.00,3528900.00,200000.0,156.0,Tandil,3427832.0,Casa venta,"Casa con 4 dormitorios. Living, Cocina, Comedo...",house
115680,Buenos Aires Interior,NaN,NaN,4.0,747.782267,-62.277651,-38.730753,75.0,ARS,56083.67,989568.31,1000000.0,70.0,Bahía Blanca,3865084.0,Oportunidad Donado 1100,Casa al frente en PH compuesta de 2 dormitorio...,house
115684,Buenos Aires Interior,NaN,NaN,5.0,172.839506,NaN,NaN,1215.0,USD,210000.00,3705345.00,210000.0,250.0,Sierra de los Padres,3427958.0,Sierras de los Padres,Chalet 5 Ambientes sobre 2 lotes de 600mts c/u...,house
115686,Buenos Aires Interior,NaN,NaN,4.0,192.307692,-58.259803,-37.837561,390.0,USD,75000.00,1323337.50,75000.0,135.0,Balcarce,3436176.0,Casa+ Depto monoambiente,"Lote de 13x30, 2 habitaciones, living comedor ...",house


In [31]:
#Obtenemos el index de todos los datos que son casas:
indices_floor_house = df.loc[df["property_type"] == "house"].index.tolist()
# print("Indices de los registros que son casas: ", indices_floor_house)
len(indices_floor_house)

37554

In [32]:
# ¿Qué porcentaje representan las casas del total de registros?
print("Porcentaje de registros que son casas del df: ", df_house.shape[0] / df.shape[0] * 100," %", sep="")

Porcentaje de registros que son casas del df: 32.46144803263951 %


In [33]:
# Imputamos todos los registros de casas para que tengan en floor, valor 0 (cero)
df.loc[df["property_type"]=="house","floor"] = 0

In [34]:
# Verificamos que haya cambiado todos los valores a cero del paso anterior.
df[["property_type", "floor"]].head(50)

,property_type,floor
0,PH,NaN
1,apartment,NaN
2,PH,NaN
3,apartment,NaN
4,apartment,NaN
5,apartment,NaN
6,apartment,NaN
7,PH,NaN
8,apartment,NaN
9,apartment,NaN


In [35]:
# Verificamos que todos los registros con "property_type" == "house", ahora tengan "floor" == 0
df[df.index.isin(indices_floor_house)].head(3)

,state_name,floor,expenses,rooms,price_usd_per_m2,lon,lat,surface_total_in_m2,currency,price_aprox_usd,price_aprox_local_currency,price,surface_covered_in_m2,place_name,geonames_id,title,description,property_type
13,Capital Federal,0.0,NaN,NaN,875.000000,-58.362056,-34.617719,80.0,USD,70000.0,1235115.0,70000.0,50.0,Puerto Madero,3429760.0,FLOAT LIFE - CASA FLOTANTE - UBICABLE EN CLUBS...,ESTRUCTURA ECOLOGICA FLOTANTEIDEAL PARA UBICAR...,house
15,Capital Federal,0.0,NaN,NaN,1091.269841,-58.410702,-34.636321,504.0,USD,550000.0,9704475.0,550000.0,324.0,Boedo,3436003.0,Casa impecable refaccionada! 6 ambientes.,Se encuentra dentro del Distrito Tecnológico. ...,house
26,Capital Federal,0.0,NaN,NaN,944.444444,NaN,NaN,180.0,USD,170000.0,2999565.0,170000.0,165.0,Capital Federal,3433955.0,"Casa 180m² con 2 Plantas en Navarro, Julián 29...","Casa en 2 Plantas en BÂ° La Merced, a Metros d...",house


In [36]:
# Vemos que luego de la imputación, bajó considerablemente la cantidad de nulos de la columna floor.
df.isnull().sum()

state_name                         0
floor                          71426
expenses                      101758
rooms                          69989
price_usd_per_m2               49058
lon                            48639
lat                            48639
surface_total_in_m2            37058
currency                       17871
price_aprox_usd                17870
price_aprox_local_currency     17870
price                          17870
surface_covered_in_m2          17744
place_name                        23
geonames_id                    18527
title                              0
description                        2
property_type                      0
dtype: int64

In [37]:
# Deseamos conocer el porcentaje de nulos por columna, vemos que el porcentaje de floor bajó considerablemente;
porcentaje_nulos_por_columna= (pd.isna(df).sum()/len(df))*100
round(porcentaje_nulos_por_columna.sort_values(ascending=False),2)

expenses                      87.96
floor                         61.74
rooms                         60.50
price_usd_per_m2              42.41
lat                           42.04
lon                           42.04
surface_total_in_m2           32.03
geonames_id                   16.01
currency                      15.45
price                         15.45
price_aprox_local_currency    15.45
price_aprox_usd               15.45
surface_covered_in_m2         15.34
place_name                     0.02
description                    0.00
property_type                  0.00
title                          0.00
state_name                     0.00
dtype: float64

In [38]:
# Analizamos cómo cambió la media, el std, y el max de floor luego de los cambios.
df.loc[(df["property_type"] == "apartment") & (df["state_name"] == "Capital Federal"), :].describe()

,floor,expenses,rooms,price_usd_per_m2,lon,lat,surface_total_in_m2,price_aprox_usd,price_aprox_local_currency,price,surface_covered_in_m2,geonames_id
count,2773.000000,6.196000e+03,13482.000000,19896.000000,19446.000000,19446.000000,22580.000000,2.415400e+04,2.415400e+04,2.415400e+04,25240.000000,2.595000e+04
mean,4.955283,6.634879e+03,2.545097,3324.464870,-58.433276,-34.599753,87.589017,2.453469e+05,4.329024e+06,3.706782e+05,79.353843,3.530139e+06
std,4.337346,1.796656e+05,1.439610,3892.824716,0.053687,0.037507,1332.550832,4.397744e+05,7.759600e+06,1.042946e+06,1183.327820,5.557376e+05
min,1.000000,1.000000e+00,1.000000,31.369688,-63.616672,-38.416097,0.000000,5.103610e+03,9.005064e+04,5.500000e+03,0.000000,3.427366e+06
25%,2.000000,1.200000e+03,1.000000,2111.111111,-58.461123,-34.619692,40.000000,9.901250e+04,1.747026e+06,1.040905e+05,37.000000,3.429760e+06
50%,4.000000,2.000000e+03,2.000000,2566.046646,-58.434144,-34.600369,56.000000,1.450000e+05,2.558452e+06,1.540000e+05,50.000000,3.433775e+06
75%,7.000000,3.900000e+03,3.000000,3168.758621,-58.402259,-34.580400,89.000000,2.500000e+05,4.411125e+06,2.890000e+05,79.000000,3.436003e+06
max,43.000000,1.000150e+07,22.000000,206333.333333,-58.354956,-34.534530,200000.000000,4.654544e+07,8.212711e+08,5.000000e+07,187000.000000,6.693230e+06


In [39]:
# Total de datos diferentes a NaN y diferentes a cero, filtrados por departamentos.
dptos_dif_nan_y_cero = df.loc[(df["floor"] != 0) & (~np.isnan(df["floor"]))& (df["property_type"] == "apartment"), :].shape

print("(dptos) Total de datos diferentes a NaN y diferentes a cero: ", dptos_dif_nan_y_cero)

(dptos) Total de datos diferentes a NaN y diferentes a cero:  (6158, 18)


In [40]:
# Cantidad total de registros de dptos en el dataset.
dptos_all_shape = df.loc[df["property_type"] == "apartment"].shape
print("Cantidad total de registros de dptos en el dataset: ", dptos_all_shape)

Cantidad total de registros de dptos en el dataset:  (68740, 18)


In [41]:
# Porcentaje de registros de dptos en todo el dataset con valores distintos de cero y distintos de NaN, respecto al total de registros de dptos.
proporcion_dptos_not_nan_ni_cero_all_dptos = dptos_dif_nan_y_cero[0] / dptos_all_shape[0] * 100

print("Relación dptos not null ni cero respecto a total dptos: ", proporcion_dptos_not_nan_ni_cero_all_dptos)

Relación dptos not null ni cero respecto a total dptos:  8.958393948210649


Decición final sobre la columna "floor": Bye.

Motivo: La proporción de departamentos que no tienen valores cero ni NaN, respecto del total es demasiado baja para tomar decisiones al respecto.

In [54]:
# drop columna floor respecto a la conclusión de arriba;
# df = df.drop(["floor"], axis=1) --> Lo comentamos porque ya lo dropeamos y devuelve error;

In [59]:
# drop columna expenses por faltante de datos, no creemos que vaya a sumar al modelo;
# df = df.drop(["expenses"], axis=1);  --> Lo comentamos porque ya lo dropeamos y devuelve error;
df.head(5)

,state_name,rooms,price_usd_per_m2,lon,lat,surface_total_in_m2,currency,price_aprox_usd,price_aprox_local_currency,price,surface_covered_in_m2,place_name,geonames_id,title,description,property_type
0,Capital Federal,NaN,1127.272727,-58.508839,-34.661824,55.0,USD,62000.0,1093959.0,62000.0,40.0,Mataderos,3430787.0,2 AMB TIPO CASA SIN EXPENSAS EN PB,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",PH
1,Capital Federal,NaN,1309.090909,-58.522982,-34.652262,55.0,USD,72000.0,1270404.0,72000.0,55.0,Mataderos,3430787.0,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,apartment
2,Capital Federal,NaN,NaN,-58.516424,-34.647797,NaN,USD,95000.0,1676227.5,95000.0,NaN,Liniers,3431333.0,PH 3 amb. cfte. reciclado,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH
3,Capital Federal,NaN,3066.666667,-58.443362,-34.559873,45.0,USD,138000.0,2434941.0,138000.0,40.0,Belgrano,3436077.0,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,apartment
4,Capital Federal,NaN,3000.000000,-58.443362,-34.559873,65.0,USD,195000.0,3440677.5,195000.0,60.0,Belgrano,3436077.0,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,apartment


In [43]:
df.groupby("state_name").sum()

,floor,expenses,rooms,price_usd_per_m2,lon,lat,surface_total_in_m2,price_aprox_usd,price_aprox_local_currency,price,surface_covered_in_m2,geonames_id
state_name,,,,,,,,,,,,
Bs.As. G.B.A. Zona Norte,7147.0,12926104.0,21661.0,3.238687e+07,-6.709070e+05,-394168.331187,5888821.0,7.802421e+09,1.376698e+11,1.101509e+10,4279033.0,5.289845e+10
Bs.As. G.B.A. Zona Oeste,2057.0,1756889.0,14329.0,6.240610e+06,-3.293698e+05,-194598.172284,1593780.0,1.228612e+09,2.167825e+10,1.410931e+09,1030931.0,2.742407e+10
Bs.As. G.B.A. Zona Sur,1118.0,4287371.0,9420.0,9.416028e+06,-3.478982e+05,-207740.771305,1639034.0,2.399546e+09,4.233879e+10,2.932530e+09,1634320.0,3.267782e+10
Buenos Aires Costa Atlántica,825.0,906538.0,22004.0,9.998920e+06,-4.714543e+05,-310316.140925,1492864.0,1.299341e+09,2.292623e+10,2.154183e+09,888034.0,3.088088e+10
Buenos Aires Interior,84.0,183488.0,3798.0,9.705104e+05,-9.348831e+04,-57073.157595,1269373.0,3.324155e+08,5.865305e+09,6.040101e+08,290854.0,7.794471e+09
Capital Federal,14165.0,41902139.0,41584.0,7.304215e+07,-1.389029e+06,-822485.221406,3151884.0,7.823345e+09,1.380390e+11,1.111251e+10,2931325.0,1.094785e+11
Córdoba,1950.0,3829720.0,16755.0,4.480915e+06,-3.291815e+05,-161139.841850,2050506.0,1.840356e+09,3.247216e+10,8.484470e+09,1283796.0,4.633996e+10
Santa Fe,2238.0,1930574.0,10017.0,8.625408e+06,-3.266505e+05,-175822.377721,997608.0,7.557069e+08,1.333407e+10,6.211581e+09,592367.0,3.897947e+10


In [44]:
df.groupby(["state_name","property_type"]).describe()

floor                        \
                                              count       mean        std   
state_name                   property_type                                  
Bs.As. G.B.A. Zona Norte     PH                14.0   1.714286   1.138729   
                             apartment       1623.0   4.380160   5.102793   
                             house          13364.0   0.000000   0.000000   
                             store             13.0   1.076923   0.277350   
Bs.As. G.B.A. Zona Oeste     PH                31.0   3.000000   2.280351   
                             apartment        535.0   3.661682   2.856678   
                             house           4663.0   0.000000   0.000000   
                             store              4.0   1.250000   0.500000   
Bs.As. G.B.A. Zona Sur       PH                24.0   2.166667   2.035910   
                             apartment        237.0   4.485232   3.496486   
                             house           6985.0   0.000000   0.000000   
                             store              2.0   1.500000   0.707107   
Buenos Aires Costa Atlántica PH                 6.0   3.000000   1.549193   
                             apartment        155.0   5.200000   4.484316   
                             house           2685.0   0.000000   0.000000   
                             store              1.0   1.000000        NaN   
Buenos Aires Interior        PH                 2.0   3.000000   2.828427   
                             apartment         21.0   3.619048   2.438774   
                             house           1325.0   0.000000   0.000000   
                             store              1.0   2.000000        NaN   
Capital Federal              PH               155.0   2.251613   2.211521   
                             apartment       2773.0   4.955283   4.337346   
                             house           2087.0   0.000000   0.000000   
                             store              8.0   9.375000  10.979689   
Córdoba                      PH               268.0   3.910448   2.959053   
                             apartment        344.0   2.587209   3.844417   
                             house           4521.0   0.000000   0.000000   
                             store              9.0   1.333333   0.500000   
Santa Fe                     PH                11.0   2.636364   1.911687   
                             apartment        470.0   4.678723   4.356816   
                             house           1924.0   0.000000   0.000000   
                             store              1.0  10.000000        NaN   

                                                                            \
                                             min    25%   50%    75%   max   
state_name                   property_type                                   
Bs.As. G.B.A. Zona Norte     PH              1.0   1.00   1.0   2.00   5.0   
                             apartment       1.0   1.00   3.0   5.00  50.0   
                             house           0.0   0.00   0.0   0.00   0.0   
                             store           1.0   1.00   1.0   1.00   2.0   
Bs.As. G.B.A. Zona Oeste     PH              1.0   1.00   2.0   4.00   9.0   
                             apartment       1.0   2.00   3.0   5.00  24.0   
                             house           0.0   0.00   0.0   0.00   0.0   
                             store           1.0   1.00   1.0   1.25   2.0   
Bs.As. G.B.A. Zona Sur       PH              1.0   1.00   1.5   2.25  10.0   
                             apartment       1.0   2.00   4.0   7.00  20.0   
                             house           0.0   0.00   0.0   0.00   0.0   
                             store           1.0   1.25   1.5   1.75   2.0   
Buenos Aires Costa Atlántica PH              2.0   2.00   2.5   3.00   6.0   
                             apartment       1.0   2.00   4.0   7.00  22.0   
                        

In [45]:
min_surface_covered_in_m2 = df["surface_covered_in_m2"].min()
print("min_surface_covered_in_m2", min_surface_covered_in_m2)

max_surface_covered_in_m2 = df["surface_covered_in_m2"].max()
print("max_surface_covered_in_m2", max_surface_covered_in_m2)

mean_surface_covered_in_m2 = df["surface_covered_in_m2"].mean()
print("mean_surface_covered_in_m2", mean_surface_covered_in_m2)

min_surface_covered_in_m2 0.0
max_surface_covered_in_m2 187000.0
mean_surface_covered_in_m2 132.02095074736584


In [46]:
#Estoy queriendo ver cuál es el dato que tiene este valor de superficie cubierta en m2 pero no estoy pudiendo dar en cómo hacerlo.
df.sort_values("surface_covered_in_m2", ascending=False)

,state_name,floor,expenses,rooms,price_usd_per_m2,lon,lat,surface_total_in_m2,currency,price_aprox_usd,price_aprox_local_currency,price,surface_covered_in_m2,place_name,geonames_id,title,description,property_type
20644,Capital Federal,NaN,11000.0,NaN,NaN,-58.409166,-34.594573,NaN,USD,600000.0,10586700.0,600000.0,187000.0,Barrio Norte,3436109.0,Excelente PISO en Barrio Norte,Barrio Norte EXCELENTE Semipiso al Frente Balc...,apartment
98185,Buenos Aires Costa Atlántica,NaN,NaN,2.0,1423.076923,-57.547773,-37.989809,52.0,USD,74000.0,1305693.0,74000.0,49000.0,Mar del Plata,3430863.0,oportunidad frente al mar!!!!,OPORTUNIDAD!!! VENTA DOS AMBIENTES FRENTE A PL...,apartment
41436,Bs.As. G.B.A. Zona Norte,0.0,NaN,NaN,516.666667,NaN,NaN,1200.0,USD,620000.0,10939590.0,620000.0,38240.0,Delta,NaN,"Casa 1200m² con 2 Plantas, Tigre, Delta, por U...","Terreno de 20mts frente x 60mts fondo, Total 1...",house
48168,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,35000.0,Tortuguitas,NaN,Nave Industrial - Tortuguitas,Plaza Logística Tortugas CÓDIGO 0703 T35.000 m...,store
70553,Bs.As. G.B.A. Zona Sur,NaN,NaN,NaN,64.516129,-58.425045,-34.781917,93000.0,USD,6000000.0,105867000.0,6000000.0,30000.0,Lomas de Zamora,3431270.0,LAVALLOL. Predio Industrial de 93.000 m2 y 30....,CODIGO: 2484-FB0007 ubicado en: Garibaldi al 2...,store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115651,Buenos Aires Interior,0.0,NaN,NaN,300.925926,-57.902854,-34.922468,432.0,USD,130000.0,2293785.0,130000.0,NaN,Berisso,3436038.0,Casa Amplia con Cochera y Dto interno con Patio,Casa en venta de 3 dormitorios uno en suite ;...,house
115656,Buenos Aires Interior,0.0,NaN,NaN,NaN,NaN,NaN,NaN,USD,220000.0,3881790.0,220000.0,NaN,Ranchos,3429610.0,Casa - Ranchos,"Casa, General Paz, Ranchos, por U$S 220.000",house
115658,Buenos Aires Interior,NaN,NaN,NaN,NaN,-62.270166,-38.712532,NaN,USD,78000.0,1376271.0,78000.0,NaN,Bahía Blanca,3865084.0,Caronti Nº 108,"Departamento en piso 10, vista al frente con ...",apartment
115677,Buenos Aires Interior,NaN,NaN,NaN,2373.913043,NaN,NaN,92.0,USD,218400.0,3853558.8,218400.0,NaN,Luján,3430982.0,Departamento - Centro De Lujan,"Departamento de tres ambientes, con dos habita...",apartment


In [47]:
df.pivot_table("price_usd_per_m2", index="state_name", columns="property_type", aggfunc=["mean", "max", "min"])

mean                            \
property_type                          PH    apartment        house   
state_name                                                            
Bs.As. G.B.A. Zona Norte      1451.712566  2539.871735  1339.764409   
Bs.As. G.B.A. Zona Oeste      1224.018825  1720.754481  1081.629587   
Bs.As. G.B.A. Zona Sur        1114.012013  1889.978782  1164.503788   
Buenos Aires Costa Atlántica  1156.454370  1958.838430   749.149782   
Buenos Aires Interior         1019.888682  1633.999387   643.236465   
Capital Federal               1695.517729  3324.464870  1546.701574   
Córdoba                       1544.048178  1426.524689   713.335035   
Santa Fe                      1207.188748  2391.993013   990.905762   

                                                    max                 \
property_type                       store            PH      apartment   
state_name                                                               
Bs.As. G.B.A. Zona Norte      1862.009755  11333.333333   48296.222222   
Bs.As. G.B.A. Zona Oeste      1556.947316  16000.000000   24500.000000   
Bs.As. G.B.A. Zona Sur        1541.024184   2800.000000   23140.764706   
Buenos Aires Costa Atlántica  1401.045662   5957.446809   21897.058824   
Buenos Aires Interior          987.670901   1867.647059    3657.142857   
Capital Federal               2899.667204  25000.000000  206333.333333   
Córdoba                       1458.775597  30000.000000   14960.000000   
Santa Fe                      3002.901146   1724.573000   44425.675676   

                                                                 min  \
property_type                        house         store          PH   
state_name                                                             
Bs.As. G.B.A. Zona Norte      38000.000000  20833.333333   49.485588   
Bs.As. G.B.A. Zona Oeste      80000.000000  11818.181818   77.893944   
Bs.As. G.B.A. Zona Sur         9411.764706  13000.000000   50.475300   
Buenos Aires Costa Atlántica   5698.005128   4740.259740   55.000000   
Buenos Aires Interior          4166.666667   2333.333333   45.824847   
Capital Federal               23064.516129  22000.000000  120.964706   
Córdoba                       16666.666667   5521.472393   36.601307   
Santa Fe                      36549.707237  57382.080000  218.103167   

                                                                 
property_type                 apartment       house       store  
state_name                                                       
Bs.As. G.B.A. Zona Norte      29.906933    4.166667  158.730159  
Bs.As. G.B.A. Zona Oeste      20.898642    7.000000   28.041833  
Bs.As. G.B.A. Zona Sur        91.135875   19.314341   22.222222  
Buenos Aires Costa Atlántica   1.000000    3.000000   82.540833  
Buenos Aires Interior         15.160342    2.072029  133.928571  
Capital Federal               31.369688  190.000000   13.797467  
Córdoba                        1.184571   12.254902   31.406853  
Santa Fe                       0.600000    9.388501  416.666667

In [48]:
df.pivot_table("price_usd_per_m2", ["state_name", "property_type"], columns="surface_covered_in_m2")

surface_covered_in_m2                           0.0          1.0      \
state_name                   property_type                             
Bs.As. G.B.A. Zona Norte     PH                     NaN          NaN   
                             apartment              NaN          NaN   
                             house                  NaN  3021.995642   
                             store                  NaN          NaN   
Bs.As. G.B.A. Zona Oeste     PH                     NaN          NaN   
                             apartment              NaN          NaN   
                             house                  NaN   712.080264   
                             store                  NaN          NaN   
Bs.As. G.B.A. Zona Sur       PH                     NaN  1046.444266   
                             apartment              NaN  1605.872247   
                             house                  NaN  1353.239659   
                             store                  NaN  1915.638724   
Buenos Aires Costa Atlántica PH                     NaN          NaN   
                             apartment              NaN          NaN   
                             house                  NaN          NaN   
                             store                  NaN          NaN   
Buenos Aires Interior        PH                     NaN          NaN   
                             apartment              NaN          NaN   
                             house                  NaN          NaN   
                             store                  NaN          NaN   
Capital Federal              PH                     NaN          NaN   
                             apartment      2467.788462          NaN   
                             house                  NaN  1662.629332   
                             store                  NaN          NaN   
Córdoba                      PH                     NaN          NaN   
                             apartment              NaN          NaN   
                             house                  NaN   306.666667   
                             store                  NaN          NaN   
Santa Fe                     PH                     NaN          NaN   
                             apartment              NaN          NaN   
                             house                  NaN          NaN   
                             store                  NaN          NaN   

surface_covered_in_m2                           2.0          3.0      \
state_name                   property_type                             
Bs.As. G.B.A. Zona Norte     PH                     NaN          NaN   
                             apartment      2380.000000          NaN   
                             house                  NaN          NaN   
                             store                  NaN          NaN   
Bs.As. G.B.A. Zona Oeste     PH                     NaN          NaN   
                             apartment              NaN          NaN   
                             house                  NaN          NaN   
                             store           680.000000          NaN   
Bs.As. G.B.A. Zona Sur       PH                     NaN          NaN   
                             apartment      1463.414634          NaN   
                             house          1346.153846          NaN   
                             store                  NaN          NaN   
Buenos Aires Costa Atlántica PH                     NaN          NaN   
                             apartment              NaN          NaN   
                             house                  NaN          NaN   
                             store                  NaN          NaN   
Buenos Aires Interior        PH                     NaN          NaN   
                             apartment              NaN          NaN   
                             house                  NaN          NaN   
                             store     

In [49]:
df.duplicated().sum()

4787